In [0]:
-- For whatever reason, part1 and part2 when ran together drives the performance super slow.
-- So the code for part1 is commented.

-- Part2 was done using A LOT cursor. My simplex knowledge from the previous millenium has vanished.
-- I could re-code the algo myself but there are so many little stuff here and there I'm not confident to proceed.
-- Still got correct result in about 15 seconds on a 2XS, I'll take that!
-- I've reviewed and corrected most of the code (not the logic) and improved the performances by 50%.


with cte_data (machine_id, tgt_lights, buttons_list, joltage, rnd) as
(
select monotonically_increasing_id() + 1
     , translate(regexp_extract(value, r'^\[(.+)\]'), '.#', '01')
     , transform(regexp_extract_all(value, r'\(([^)]+)\)', 1), t -> array_sort(transform(split(t, ','), u -> u::int)))
     , transform(split(regexp_extract(value, r'\{(.+)\}$'), ','), t -> t::int)
     , rand()
  from read_files('/Volumes/waldar/fabien/aoc2025/aoc2025_10_input.txt', format => 'text', wholeText => false) as t (value)
/*from values ('[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}')
            , ('[...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}')
            , ('[.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}') as t (value)*/
)
  ,  cte_prep (machine_id, tgt_b10, buttons_b10, inc, joltage, max_val, num_rows, num_vars, mat) as
(
select machine_id
     , conv(tgt_lights, 2, 10)::int
     , transform(buttons_list, t -> aggregate(t, 0, (acc, x) -> acc + (1 << (length(tgt_lights) - x - 1))))
     , transform(buttons_list, b -> aggregate(b, 0, (acc, x) -> acc + power(10, length(tgt_lights) - x - 1)::int, acc -> transform(split(lpad(acc, length(tgt_lights), '0'), ''), u -> u::int))) as inc
     , joltage
     , array_max(joltage)::double
     , size(joltage) as num_rows
     , size(inc)     as num_vars
     , transform( sequence(0, num_rows - 1)
                , t -> array_append(transform(sequence(0, num_vars - 1), u -> inc[u][t]::double), joltage[t]::double)) mat
  -- , transform(inc, t -> array_min(filter(transform(arrays_zip(joltage, t), u -> u.joltage * u.t), f -> f > 0))) as ai
  from cte_data
 where true
)
/*
  ,  cte_part1 as
(
select sum(aggregate( sequence(1,10)
                    , struct(0 as n, array(0) as s)
                    , (acc, n) -> case size(filter(acc.s, f -> f = tgt_b10))
                                    when 0
                                    then struct(n, array_distinct(flatten(transform(acc.s, a -> transform(buttons_b10, b -> a ^ b)))) as s)
                                    else acc
                                  end
                    , acc -> acc.n
                    ) ) as part1
  from cte_prep
)
select part1 from cte_part1;
*/
  ,  cte_forward (machine_id, reff, num_rows, num_vars, max_val) as
(
select machine_id
     , aggregate( sequence(0, least(num_rows, num_vars) - 1)
                , mat
                , (m, x) -> aggregate( array()::array<int>
                                     , coalesce(filter(sequence(x, size(m)-1), f -> m[f][x] <> 0)[0], x)
                                     , (azz, z) -> null
                                     , azz -> transform( m
                                                       , (t, i) -> case
                                                                     when i < x 
                                                                     then t
                                                                     when i = x
                                                                     then m[azz]
                                                                     when i > x
                                                                      and i = azz
                                                                     then m[x]
                                                                     when i > x
                                                                      and m[azz][x] = 0
                                                                     then t
                                                                     else transform(t, (u, j) -> u - (t[x] / m[azz][x]) * m[azz][j])
                                                                   end )
                                     )
                )
     , num_rows
     , num_vars
     , max_val
  from cte_prep
)
  ,  cte_pivots (machine_id, num_rows, num_vars, max_val, reff, fnz, ref, pcr) as
(
select machine_id, num_rows, num_vars, max_val, reff
     , transform(reff, (r,i) -> coalesce(array_min(transform(r, (t,j) -> case when abs(t) > 0.0001d then j end)), -1)) as fnz
     , aggregate( sequence(0, least(num_rows, 9) - 1)
                , reff
                , (m, p) -> transform(m, (t, i) -> case
                                                     when i <= p
                                                       or fnz[i] < 0
                                                       or fnz[i] <> fnz[p]
                                                       or abs(m[p][fnz[p]]) < 0.0001d
                                                     then t
                                                     else transform(t, (u, j) -> u - (t[fnz[p]] / m[p][fnz[p]]) * m[p][j])
                                                    end)
                ) as ref
     , transform(ref, (r,i) -> coalesce(array_min(transform(r, (t,j) -> case when abs(t) > 0.1 then j end)), -1)) as pcr
  from cte_forward
)
  ,  cte_rref (machine_id, num_vars, max_val, pivot_cols, valid_rows, rref) as
(
select machine_id
     , num_vars
     , max_val
     , transform(sequence(0, size(pcr) - 1), r -> case when r <> coalesce(filter(sequence(0, size(pcr) - 1), i -> pcr[i] = pcr[r])[0], -1) then -1 else pcr[r] end) as pivot_cols
     , filter(sequence(0, size(pcr) - 1), r -> pcr[r] >= 0 and r = coalesce(filter(sequence(0, size(pcr) - 1), i -> pcr[i] = pcr[r])[0], -1)) as valid_rows
     , aggregate( filter(sequence(0, size(pcr) - 1), r -> pcr[r] >= 0 and r = coalesce(filter(sequence(0, size(pcr) - 1), i -> pcr[i] = pcr[r])[0], -1))
                , ref
                , (m, r) -> transform( m
                                     , (t, i) -> case i
                                                     when r
                                                     then transform(t,  u     -> case when abs(m[r][pcr[r]]) < 0.0001d then u else u / m[r][pcr[r]] end)
                                                     else transform(t, (u, j) -> case when abs(m[r][pcr[r]]) < 0.0001d then u else u - (t[pcr[r]] / m[r][pcr[r]]) * m[r][j] end)
                                                   end
                                     )
                ) as rref
  from cte_pivots
)
  ,  cte_bounds (machine_id, rref, num_vars, max_val, pivot_cols, valid_rows, free_vars, bounds) as
(
select machine_id, rref, num_vars, max_val, pivot_cols, valid_rows
     , array_except(sequence(0, num_vars - 1), pivot_cols) as free_vars
     , transform( free_vars
                , fv -> struct( aggregate( sequence(0, size(rref) - 1)
                                         , max_val
                                         , (ub, r) -> case 
                                                        when coalesce(rref[r][fv]      , 0d) > 0.01d
                                                         and coalesce(rref[r][num_vars], 0d) > 0d
                                                         and size(filter( free_vars
                                                                        , fv2 -> fv2 <> fv
                                                                             and coalesce(rref[r][fv2], 0d) < -0.01d)) = 0
                                                        then least(ub, rref[r][num_vars] / rref[r][fv])
                                                        else ub
                                                      end
                                          , ub -> floor(ub)::int
                                          ) as u
                              , aggregate( sequence(0, size(rref) - 1)
                                         , 0d
                                         , (lb, r) -> case 
                                                        when coalesce(rref[r][fv]      , 0d) < 0.01d
                                                         and coalesce(rref[r][num_vars], 0d) < 0d
                                                         and size(filter( free_vars
                                                                        , fv2 -> coalesce(rref[r][fv2], 0d) < -0.01d)) = 1
                                                        then greatest(lb, rref[r][num_vars] / rref[r][fv])
                                                        else lb
                                                      end
                                          , lb -> ceiling(lb)::int
                                          ) as l
                              )
                ) as bounds
  from cte_rref
)
  ,  cte_ranges (machine_id, rref, num_vars, pivot_cols, valid_rows, free_vars, r0, r1, r2, combos) as
(
select machine_id, rref, num_vars, pivot_cols, valid_rows, free_vars
     , case when size(bounds) >= 1 and bounds[0].l <= bounds[0].u then sequence(bounds[0].l, bounds[0].u) else array(0) end as r0
     , case when size(bounds) >= 2 and bounds[1].l <= bounds[1].u then sequence(bounds[1].l, bounds[1].u) else array(0) end as r1
     , case when size(bounds) >= 3 and bounds[2].l <= bounds[2].u then sequence(bounds[2].l, bounds[2].u) else array(0) end as r2
     , case size(free_vars)
         when 0
         then array(array()::array<int>)
         when 1
         then transform(r0, t -> array(t))
         when 2
         then flatten(transform(r0, t -> transform(r1, u -> array(t, u))))
         when 3
         then flatten(flatten(transform(r0, t -> transform(r1, u -> transform(r2, v -> array(t, u, v))))))
       end
  from cte_bounds
)
  ,  cte_combos (machine_id, rref, num_vars, free_vars, pivot_cols, valid_rows, combo) as
(
select machine_id, rref, num_vars, free_vars, pivot_cols, valid_rows, combo
  from cte_ranges
  join lateral explode(combos) as u (combo)
)
  , cte_solved (machine_id, sol) as
(
select machine_id
     , aggregate( reverse(valid_rows)
                , transform( sequence(0, num_vars - 1)
                           , var -> case when array_contains(free_vars, var) then combo[array_position(free_vars, var) - 1]::double else -9999d end )
                , (sol, r) -> transform( sol
                                       , (v, var) -> case var
                                                       when pivot_cols[r]
                                                       then coalesce(rref[r][num_vars], 0d)
                                                          - aggregate( sequence(0, num_vars - 1)
                                                                     , 0d
                                                                     , (s, c) -> case
                                                                                   when c = var
                                                                                     or sol[c] < -9000d
                                                                                   then 0d
                                                                                   else coalesce(rref[r][c], 0d) * sol[c]
                                                                                 end + s
                                                                     )
                                                       else v
                                                     end )
                )
  from cte_combos
)
  ,  cte_valid (machine_id, solution, total_cost) as
(
 select machine_id
      , transform(sol, v -> round(v)::int) -- kept for controls
      , aggregate(sol, 0, (a, v) -> a + round(v)::int) as total_cost
   from cte_solved
  where aggregate(sol, true, (ok, v) -> ok and v >= -0.01d and v < 9000d and abs(v - round(v)) < 0.1d)
 window W as (partition by machine_id
                  order by total_cost asc)
qualify row_number() over W = 1
)
select sum(total_cost) as part2
  from cte_valid
;

